In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [34]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
from src.models.siamese_modeling import SiameseDataSet, SiameseNetwork
from torch.utils.tensorboard import SummaryWriter

from src.models.DenseNetwork import loss

from tqdm.auto import tqdm
import copy

torch.manual_seed(0)

In [3]:
train_dataset = SiameseDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/train.csv')
val_dataset = SiameseDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/dev.csv')

In [4]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1000, pin_memory=True)

In [5]:
weight_decay = 1e-5
learning_rate = 1e-5
num_epoches = 40

writer = SummaryWriter('runs/siamese_network')

In [6]:
device = torch.device('cuda:0')
model = SiameseNetwork()
model = model.to(device)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)


In [7]:
def train_one_epoch(train_loader, model, optimizer, verbose):
    model = model.to(device)
    model.train()
    loss_sum = 0.
    for i, batch in enumerate(train_loader):
        x1, x2, labels = batch
        x1, x2, labels = x1.to(device), x2.to(device), labels.to(device)
        dist, loss = model(x1, x2, labels)
        model.zero_grad()  # reset gradient
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        if verbose and i % 20 == 0:
            print(f'training loss: {loss_sum / (i + 1):.4f}')
    return loss_sum / len(train_loader)

def val_one_epoch(val_loader, model):
    model.eval()
    val_contrastive_loss = 0.
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            x1, x2, labels = batch
            x1, x2, labels = x1.to(device), x2.to(device), labels.to(device)
            dist, loss = model(x1, x2, labels)
            val_contrastive_loss += loss.item()
    return val_contrastive_loss / len(val_loader)

In [8]:
def train_with_eval(train_loader, val_loader, model, optimizer, num_epoches, verbose):
    global writer
    best_model = None
    best_avg_val_contrastive_loss = float('inf')
    for epoch_idx in range(1, num_epoches + 1):
        avg_loss = train_one_epoch(train_loader, model, optimizer, False)
        avg_val_contrastive_loss = val_one_epoch(val_loader, model)
        if avg_val_contrastive_loss <  best_avg_val_contrastive_loss:
            best_avg_val_contrastive_loss = avg_val_contrastive_loss
            best_model = copy.deepcopy(model.cpu())
        writer.add_scalar('train/avg_mixed_loss', avg_loss, epoch_idx)
        writer.add_scalar('val/avg_val_contrastive_loss', avg_val_contrastive_loss, epoch_idx)
        if verbose and epoch_idx % 4 == 0:
            print(f'epoch [{epoch_idx}]/[{num_epoches}] training loss: {avg_loss:.4f} '
                  f'avg_val_contrastive_loss: {avg_val_contrastive_loss:.4f} ')
    return best_avg_val_contrastive_loss, best_model, model

In [9]:
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1000, pin_memory=True)

In [10]:
best_avg_val_contrastive_loss, best_model, final_model = train_with_eval(train_loader, val_loader, model, optimizer, num_epoches, True)


epoch [4]/[40] training loss: 0.0893 avg_val_contrastive_loss: 0.1769 
epoch [8]/[40] training loss: 0.0650 avg_val_contrastive_loss: 0.1839 
epoch [12]/[40] training loss: 0.0487 avg_val_contrastive_loss: 0.1916 
epoch [16]/[40] training loss: 0.0360 avg_val_contrastive_loss: 0.1992 
epoch [20]/[40] training loss: 0.0258 avg_val_contrastive_loss: 0.2019 
epoch [24]/[40] training loss: 0.0178 avg_val_contrastive_loss: 0.2033 
epoch [28]/[40] training loss: 0.0125 avg_val_contrastive_loss: 0.2041 
epoch [32]/[40] training loss: 0.0098 avg_val_contrastive_loss: 0.2069 
epoch [36]/[40] training loss: 0.0078 avg_val_contrastive_loss: 0.2137 
epoch [40]/[40] training loss: 0.0067 avg_val_contrastive_loss: 0.2128 


In [14]:
# torch.save({
#     "best_model": best_model.state_dict(),
#     "best_avg_val_contrastive_loss": best_avg_val_contrastive_loss
# }, '../saves/siamese.on.full.0.0067')

In [18]:
def extract_embeddings(data_loader, model):
    model.eval()
    embedding = list()
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            x = batch[0].to(device)
            out = model.encode_batch(x)
            embedding.append(out.cpu())
    return torch.cat(embedding, dim=0)
best_model = best_model.to(device)
val_nn_loader = DataLoader(TensorDataset(val_dataset.data.clone()), batch_size=1000, pin_memory=True)
val_x_embedded = extract_embeddings(val_nn_loader, best_model)

In [55]:
class RetrieveSystem(object):
    def __init__(self, distance_measure):
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        distance_measure = distance_measure.to(self.device)
        self.distance_measure = distance_measure

    def retrieve_query(self, query, ignore_idx, x_embedded, x_idx, topk=20):
        query_device = query.view(1, -1).to(self.device)
        cls_distances = list()
        with torch.no_grad():
            for i, x in zip(x_idx, x_embedded):
                if ignore_idx is not None and i == ignore_idx:
                    continue
                x_device = x.view(1, -1).to(self.device)
                logits = self.distance_measure.decode_batch(query_device, x_device)
                cls_distances.append(logits.item())
                
        cls_distances = torch.tensor(cls_distances)
        _, cls_nn_idx = cls_distances.sort(descending=False)
        return cls_nn_idx[:topk]

    def retrieve_corpus(self, corpus, block_list, database):
        cls_pred_nn_top = list()
        x_idx = range(database.shape[0])
        for ignore_idx, query in tqdm(zip(block_list, corpus), total=len(block_list), desc='retrieve each query'):
            cls_distances = self.retrieve_query(query, ignore_idx, database, x_idx, 20)
            cls_pred_nn_top.append(cls_distances.view(1, -1))
        cls_pred_nn_top = torch.cat(cls_pred_nn_top, dim=0)
        return cls_pred_nn_top

    def recall(self, pred, gold, at_n=None):
        results = dict()
        if at_n is None:
            at_n = [1, 5, 10, 20]
        for n in at_n:
            recall = float((pred[:, :n] == gold.view(-1, 1)).sum().item()) / len(gold)
            results[f'recall@{n}'] = recall
        return results


In [56]:
retriever = RetrieveSystem(best_model)
block_list = torch.arange(val_x_embedded.shape[0])
cls_pred_nn_top = retriever.retrieve_corpus(val_x_embedded, block_list, val_x_embedded)


In [57]:

_, gold, _ = loss.nearest_neighbors(val_dataset.data.clone(), top_k=1, device='cuda')

In [58]:
cls_pred_nn_top.shape

torch.Size([1000, 20])

In [59]:
len(gold)

1000

In [60]:
float((cls_pred_nn_top[:, :5] == gold.view(-1, 1)).sum()) / 1000

0.038